In [1]:
# IMPORT LIBS
import pandas as pd
import numpy as np
import time
import os
from glob import glob
from datetime import datetime
from logzero import logger
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict

import timescaledb_model as tsdb
db = tsdb.TimescaleStockMarketModel('bourse', 'ricou', 'localhost', 'monmdp')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

CSS = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(CSS))

Logs of timescaledb_model go to /tmp/bourse.log


In [2]:
def parse_filename(filename):
    return datetime.strptime(filename[-30:-4], "%Y-%m-%d %H:%M:%S.%f")

def erase_c(df: pd.DataFrame):
    df['last'] = df['last'].astype(str)
    df['last'] = df['last'].str.replace('(c)', '')
    df['last'] = df['last'].str.replace('(s)', '')
    df['last'] = df['last'].str.replace(' ', '')
    df['last'] = df['last'].astype(float)
    return df

def custom_sort_key(file_path: str):
    return file_path[-22:-4]

In [3]:
def get_market_id(market_name: str) -> int:
    get_id = db.raw_query(f"SELECT id FROM markets WHERE alias = '{market_name}'")
    return int(get_id[0][0]) if len(get_id) != 0 else None

def get_companies_id():
    return db.df_query('SELECT id, symbol FROM companies', chunksize=None)

In [4]:
companies_dict = defaultdict(set)

def compute_companies(df, stock_name, market_id, market_name):
    companies_df = df[['name','symbol']].copy().drop_duplicates()
    companies_df.reset_index(drop=True, inplace=True)
    companies_df['mid'] = market_id

    companies_df['pea'] = market_name == "peapme"
    companies_df['boursorama'] = stock_name == "boursorama"
    companies_df.loc[:, ["symbol_nf", "isin", "reuters", "sector"]] = None

    grouped = set(zip(companies_df["symbol"], companies_df["name"]))

    if market_name == "peapme":
        keys = "('" + "', '".join([symbol for symbol, _ in grouped]) + "')"
        db.execute(f"UPDATE companies SET pea = True WHERE symbol in {keys};")

    before_filter = len(companies_df.symbol)

    # Create the filter
    filter_list = []
    for symbol, name in grouped:
        if symbol in companies_dict:
            filter_list.append((symbol, name))

        companies_dict[symbol].add(name)

    # Applying the filter
    companies_df["tmp_filter"] = list(zip(companies_df["symbol"], companies_df["name"]))
    companies_df = companies_df[~companies_df["tmp_filter"].isin(filter_list)]
    companies_df.drop(columns=["tmp_filter"], inplace=True)

    # Write inside table
    logger.debug(f"before: {before_filter} && after: {len(companies_df.symbol)} applying filter")
    db.df_write(companies_df, "companies", commit=True, index=False)

def compute_stocks(df, compagnies_df): # GOOD A PRIORI
    stock_df = df[['symbol', 'last', 'volume']].copy().drop_duplicates()
    stock_df.reset_index(names='date', inplace=True)

    stock_df = stock_df.merge(compagnies_df, left_on='symbol', right_on='symbol', how='left')
    stock_df.drop(columns=['symbol'], inplace=True)
    stock_df.rename(columns={'last': 'value', 'id': 'cid'}, inplace=True)
    stock_df['cid'] = stock_df['cid'].fillna(0)

    # Write inside table
    db.df_write(stock_df, 'stocks', commit=True, index=False)

def compute_daystocks(df, compagnies_df): #TODO: Drop duplicate??
    daystocks_df = df.copy()

    # Convert the index to datetime if it's not already in datetime format
    daystocks_df.reset_index(names="timestamp", inplace=True)

    # Group by 'name' column
    grouped = daystocks_df.groupby(['symbol', daystocks_df['timestamp'].dt.date])

    # Apply aggregations
    daystocks_df = grouped.agg({
        'last': ['last', 'first', "min", "max"],
        'volume': "first",
    })
    daystocks_df.reset_index(inplace=True)

    # Rename columns for better clarity
    daystocks_df.columns = ['symbol', 'date', 'open', 'close', 'low', 'high', 'volume']

    daystocks_df = daystocks_df.merge(compagnies_df, left_on='symbol', right_on='symbol', how='left')
    daystocks_df.drop(columns=['symbol'], inplace=True)
    daystocks_df.rename(columns={'id': 'cid'}, inplace=True)
    daystocks_df['cid'] = daystocks_df['cid'].fillna(0)

    # Write inside table
    db.df_write(daystocks_df, "daystocks", commit=True, index=False)

In [5]:
def store_file(files, website, market_name, market_id):
    if website.lower() != "boursorama":
        return

    data = {}
    start = time.time()
    for filename in files:
        #if db.is_file_done(filename): # Useless for now
        #    continue

        # Add in final fidctionary
        data[parse_filename(filename)] = pd.read_pickle(filename)
    logger.debug("END read files", time.time() - start)

    # CONCAT FINAL DATA
    start = time.time()
    df = pd.concat(data).reset_index(level=1, drop=True)
    logger.debug("END concat files", time.time() - start)
    #display(df)

    # CLEAN DATA
    df.drop_duplicates(inplace=True)
    df.index = pd.to_datetime(df.index)
    erase_c(df)
    overflow_mask = df['volume'] > 2147483647
    df.loc[overflow_mask, 'volume'] = np.iinfo(np.int32).max

    start = time.time()
    compute_companies(df, website, market_id, market_name)
    logger.debug("END compute_companies", time.time() - start)

    start = time.time()
    compagnies_df = get_companies_id()
    #display(compagnies_df)
    logger.debug("END get_companies_id", time.time() - start)

    start = time.time()
    compute_stocks(df, compagnies_df)
    logger.debug("END compute_stocks", time.time() - start)

    start = time.time()
    compute_daystocks(df, compagnies_df)
    logger.debug("END compute_daystocks", time.time() - start)

    # Write the files as processed
    start = time.time()
    files_df = pd.DataFrame({'name': files})
    #display(files_df)
    db.df_write(files_df,"file_done",index=False)
    logger.debug("END file_done", time.time() - start)


In [6]:
# MAIN ARGS
files_count = 0
data_path = "../docker/data"


# MAIN PROCESS
logger.debug(f"Work in progress...")
starting_time = time.time()
for stock in os.listdir(data_path):
    stock_path = f"{data_path}/{stock}"

    for year in sorted(os.listdir(stock_path), reverse=True):
        market_pattern = f"{data_path}/{stock}/{year}/*"

        available_files = glob(market_pattern)
        available_files = sorted(available_files, key=custom_sort_key)

        market_files = defaultdict(lambda: defaultdict(list))

        # Organize files into separate lists based on the market name
        for file_path in available_files:
            market_name, date, _ = os.path.basename(file_path[:-4]).split()
            month = date.split('-')[1]
            market_files[market_name][month].append(file_path)

        # Sort days inside
        for market_name, month_dict in market_files.items():
            for file_list in month_dict.values():
                file_list.sort(reverse=True, key=custom_sort_key)

        # Work on the data with batches
        for market_name, month_dict in sorted(market_files.items()):
            logger.debug(f"Loading {market_name} ({year}) data for {stock}.")
            market_starting_time = time.time()

            market_id = get_market_id(market_name)
            for month, files_list in tqdm(sorted(month_dict.items(), reverse=True)):
                files_count += len(files_list)

                store_file(files_list, stock, market_name, market_id)

            subprocessing_time = round(time.time() - market_starting_time, 3)
            logger.debug(f"Processing complete for {market_name} data in {year} in"
                         f" {subprocessing_time} seconds.")

rounded_processing_time = round(time.time() - starting_time, 3)
logger.debug(f"Work done on {files_count} files, in {rounded_processing_time} seconds.")

[D 240331 22:58:32 2419859425:7] Work in progress...
[D 240331 22:58:32 2419859425:33] Loading amsterdam (2023) data for boursorama.
  0%|          | 0/12 [00:00<?, ?it/s]

END read files 23.817959308624268
END concat files 1.3170194625854492


[D 240331 22:58:58 1716370485:35] before: 4375 && after: 4355 applying filter


END compute_companies 0.653350830078125
END get_companies_id 0.007628917694091797
END compute_stocks 26.04205060005188


  8%|▊         | 1/12 [01:01<11:14, 61.27s/it]

...

END compute_daystocks 0.4004173278808594
END file_done 0.04664778709411621
END read files 3.7371485233306885
END concat files 0.3185551166534424


[D 240401 00:27:15 1716370485:35] before: 299 && after: 1 applying filter


END compute_companies 0.04501008987426758
END get_companies_id 0.008353233337402344
END compute_stocks 4.865925550460815


100%|██████████| 12/12 [01:38<00:00,  8.19s/it]
[D 240401 00:27:20 2419859425:43] Processing complete for compB data in 2019 in 98.281 seconds.
[D 240401 00:27:20 2419859425:47] Work done on 271325 files, in 5328.774 seconds.


END compute_daystocks 0.41930413246154785
END file_done 0.048840999603271484


In [7]:
"""show_data = ["low", "high"]
def plot_daystocks(dataframe):
    copy = dataframe.copy()
    copy.plot(y=show_data, figsize=(10,6), title=f"Cours de {companie_focus} pour {available_years}")
    plt.xlabel("Date")
    plt.ylabel("Cours en Euro (€)")
    plt.show()

plot_daystocks(result_daystocks)"""

'show_data = ["low", "high"]\ndef plot_daystocks(dataframe):\n    copy = dataframe.copy()\n    copy.plot(y=show_data, figsize=(10,6), title=f"Cours de {companie_focus} pour {available_years}")\n    plt.xlabel("Date")\n    plt.ylabel("Cours en Euro (€)")\n    plt.show()\n\nplot_daystocks(result_daystocks)'